In [ ]:
import tweepy
import json
import pandas as pd
import sys
from tweepy import Stream
import streamer

## Setting the Authorization

In [ ]:
# Load your own keys 
with open("twitter-keys.json") as file:
  keysjson = json.load(file)


#1. Get a request keys from twitter API, just saved in a JSON for privacy.
# Token as variables
accessToken = keysjson["access_token"]
accessTokenSecret = keysjson["access_token_secret"]
consumerKey = keysjson["consumer_key"]
consumerSecret = keysjson["consumer_secret"]
bearerToken = keysjson["bearer_token"]

#2. Authorize Access to the API using .OAuthHandler().
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

#3. Set the Access tokens.
authenticate.set_access_token(accessToken, accessTokenSecret)

#4. Create the API object while passing in the auth information.
api = tweepy.API(authenticate, wait_on_rate_limit = True) # Whether or not to automatically wait for rate limits to replenish, in case in case i run out of rquest chances

## Creating the Listener

In [ ]:
# First Stream fetched 35 tweets per second
stream = streamer.Listener(consumerKey=consumerKey,consumerSecret=consumerSecret,
                  accessToken=accessToken,accessTokenSecret=accessTokenSecret,
                  output_file="stream_output.txt")

try:
    # Start Listening 
    print('>>> Streaming Started <<<')

    # TWEEPY STREAMING QUERY 
    stream.filter(track=['covid'])
    
except KeyboardInterrupt as e :
    # Stop on interruption
    print(">>>> Streaming Stopped <<<<")
    
finally:
    tweets = streamer.Wrangler(tweets_list=stream.tweets,
                                    retweets_list=stream.retweets,
                                    quoted_list=stream.quoted,
                                    columns=stream.sub_columns).tweets
    
    # Set Date as index
    tweets = tweets.set_index('created_at')

    # Single output txt generated and a counter of tweets
    TextOut = stream.TextOut
    count_status = stream.status_counter

    
    print('>>> Streaming Finished <<<')
    stream.disconnect()